**Importing Libraries**

In [93]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_excel('/Users/amardeepsingh/Desktop/Online Retail.xlsx')

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


**Missing Values**

In [4]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
df.dropna(inplace=True)

**Data : UCI Open ML Database : Groceries** > 
Cleaning and Preprocessing

In [6]:
df.shape

(406829, 8)

In [97]:
df= df[df.Country == 'France']

In [98]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583.0,France
27,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,12583.0,France
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583.0,France
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,2010-12-01 08:45:00,0.85,12583.0,France
30,536370,21883,STARS GIFT TAPE,24,2010-12-01 08:45:00,0.65,12583.0,France


In [10]:
# Removing spaces from begginings of descriptions
df['Description'] = df['Description'].str.strip()
# Some Transanction quantities are -ve. They Will be Removed.
df=df[df.Quantity >0]

In [32]:
#Converting the data into suitable format for association rule discovery
transactions=pd.pivot_table(data=df,index='InvoiceNo',columns='StockCode',values='Quantity', aggfunc='sum',fill_value=0)

In [33]:
transactions.head()

StockCode,10002,10120,10125,10135,11001,15036,15039,16012,16048,16218,...,85232D,90030B,90030C,90184B,90184C,90201B,90201C,C2,M,POST
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,48,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


**Function to Read the Product Name**

In [34]:
def check_id(df, stockcode):
    product_name = df[df["StockCode"] == stockcode]["Description"].unique()[0]
    return stockcode, product_name

In [40]:
check_id(df,10120)

(10120, 'DOGGY RUBBER')

**Function to Read StockCode from Description**


In [37]:
def check_Description(df,product_description):
    Scode = df[df["Description"] == product_description]["StockCode"].unique()[0]
    return product_description, Scode

In [38]:
check_Description(df,'POSTAGE')

('POSTAGE', 'POST')

In [41]:
# Function to convert sums of Quantity to Binary (0,1)
def binary_converter(x):
    if x > 0:
        return 1
    else:
        return 0

In [42]:
transactions_set = transactions.applymap(binary_converter)

In [44]:
# Removing POSTAGE Item as it is not Relevant
transactions_set.drop(columns=['POST'],inplace=True)

**Apriori Function With Minimum Support = 0.7**

In [45]:
frequent_itemsets = apriori(transactions_set, min_support=0.07, use_colnames=True)

/Users/amardeepsingh/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [46]:
frequent_itemsets

,support,itemsets
0,0.077121,(20712)
1,0.074550,(20719)
2,0.097686,(20724)
3,0.154242,(20725)
4,0.118252,(20726)
5,0.138817,(20750)
6,0.133676,(21080)
7,0.138817,(21086)
8,0.128535,(21094)
9,0.077121,(21121)


**Association Rules from Frequent Itemsets**

In [47]:
# Based on Lift, Minimum Lift = 1
rules_mlxtend =association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules_mlxtend.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(21080),(21086),0.133676,0.138817,0.102828,0.769231,5.541311,0.084271,3.731791
1,(21086),(21080),0.138817,0.133676,0.102828,0.740741,5.541311,0.084271,3.341535
2,(21080),(21094),0.133676,0.128535,0.102828,0.769231,5.984615,0.085646,3.776350
3,(21094),(21080),0.128535,0.133676,0.102828,0.800000,5.984615,0.085646,4.331620
4,(21086),(21094),0.138817,0.128535,0.123393,0.888889,6.915556,0.105550,7.843188


 **Sorting Rules on the Basis of Lift and Confidence**

In [48]:
# According to Business Case, we can sort by Lift, Confidence
rules_mlxtend[ (rules_mlxtend['lift'] >= 4) & (rules_mlxtend['confidence'] >= 0.8) ].tail()
rules_mlxtend.rename(columns={'antecedents':'LHS','consequents':'RHS'})

,LHS,RHS,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(21080),(21086),0.133676,0.138817,0.102828,0.769231,5.541311,0.084271,3.731791
1,(21086),(21080),0.138817,0.133676,0.102828,0.740741,5.541311,0.084271,3.341535
2,(21080),(21094),0.133676,0.128535,0.102828,0.769231,5.984615,0.085646,3.776350
3,(21094),(21080),0.128535,0.133676,0.102828,0.800000,5.984615,0.085646,4.331620
4,(21086),(21094),0.138817,0.128535,0.123393,0.888889,6.915556,0.105550,7.843188
5,(21094),(21086),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563
6,(22554),(22551),0.172237,0.136247,0.105398,0.611940,4.491411,0.081932,2.225826
7,(22551),(22554),0.136247,0.172237,0.105398,0.773585,4.491411,0.081932,3.655955
8,(22556),(22551),0.169666,0.136247,0.089974,0.530303,3.892224,0.066858,1.838958
9,(22551),(22556),0.136247,0.169666,0.089974,0.660377,3.892224,0.066858,2.444873


**Recommending Products**

In [76]:
# Sorting by Lift 
sorted_rules = rules_mlxtend.sort_values("lift", ascending=False)

In [82]:
recom_seed=21094

# Recommendation Engine
recommendation_list = []
for idx, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == recom_seed:
            recommendation_list.append(list(sorted_rules.iloc[idx]["consequents"])[0])
            recommendation_list = list( dict.fromkeys(recommendation_list) )
list_top5 = recommendation_list[0:5]
list_top5

[21080, 21086]

In [83]:
# Descriptions of Recommendations
for elem in list_top5:
    print(check_id(df,elem))

(21080, 'SET/20 RED RETROSPOT PAPER NAPKINS')
(21086, 'SET/6 RED SPOTTY PAPER CUPS')
